<a href="https://colab.research.google.com/github/shengjiyang/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [2]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2020-03-11 21:33:13--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [                <=> ]  21.71M   781KB/s    in 29s     

2020-03-11 21:33:43 (771 KB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22763850]



In [3]:
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [12]:
import pandas as pd
df_LoanStats = pd.read_csv('LoanStats_2018Q4.csv',
                           skiprows = 1,
                           skipfooter = 2,
                           engine = 'python')

print(df_LoanStats.head())

   id  member_id  ...  settlement_percentage  settlement_term
0 NaN        NaN  ...                    NaN              NaN
1 NaN        NaN  ...                    NaN              NaN
2 NaN        NaN  ...                    NaN              NaN
3 NaN        NaN  ...                    NaN              NaN
4 NaN        NaN  ...                    NaN              NaN

[5 rows x 144 columns]


In [13]:
# Convert the term column from string to integer

print(df_LoanStats['term'])

term = '60 months'
print(term.strip(' months'))
print(int(term.strip(' months')))

stripped_term = int(term.strip(' months'))
type(stripped_term)

0          60 months
1          36 months
2          36 months
3          60 months
4          60 months
             ...    
128279     36 months
128280     36 months
128281     36 months
128282     36 months
128283     36 months
Name: term, Length: 128284, dtype: object
60
60


int

In [14]:
def clean_term(term):
    return int(term.strip(' months'))

df_LoanStats['term'] = df_LoanStats['term'].apply(clean_term)


# 成功了！
print(df_LoanStats['term'].head())
print(df_LoanStats['term'].dtypes)


0    60
1    36
2    36
3    60
4    60
Name: term, dtype: int64
int64


In [15]:
# loan_status_is_great

print(df_LoanStats['loan_status'].value_counts())



def loan_status_is_great(loan_status):
    if loan_status == 'Current' or loan_status == 'Fully Paid':
        return 1
    else:
        return 0


test_loan_status_1 = 'Current'
test_loan_status_2 = 'Fully Paid'
test_loan_status_3 = 'Late (31-120) days'
test_loan_status_4 = 'In Grace Period'
test_loan_status_5 = 'Late (16-30 days)'
test_loan_status_6 = 'Default'

Current               92485
Fully Paid            24821
Charged Off            6766
Late (31-120 days)     2210
In Grace Period        1405
Late (16-30 days)       486
Default                 111
Name: loan_status, dtype: int64


In [16]:
print([loan_status_is_great(test_loan_status_1), loan_status_is_great(test_loan_status_2),
       loan_status_is_great(test_loan_status_3), loan_status_is_great(test_loan_status_4),
      loan_status_is_great(test_loan_status_5), loan_status_is_great(test_loan_status_6)])

[1, 1, 0, 0, 0, 0]


In [20]:
df_LoanStats['loan_status_is_great'] = df_LoanStats['loan_status'].apply(loan_status_is_great)

df_LoanStats_loan_status = df_LoanStats[['loan_status',
                                         'loan_status_is_great']].copy()

print(df_LoanStats_loan_status)


               loan_status  loan_status_is_great
0               Fully Paid                     1
1                  Current                     1
2               Fully Paid                     1
3       Late (31-120 days)                     0
4                  Current                     1
...                    ...                   ...
128279         Charged Off                     0
128280             Current                     1
128281             Current                     1
128282             Current                     1
128283             Current                     1

[128284 rows x 2 columns]


In [18]:
# # Make last_pymnt_d_month and last_pymnt_d_year columns.

print(df_LoanStats['last_pymnt_d'])

df_LoanStats['last_pymnt_d'] = pd.to_datetime(df_LoanStats['last_pymnt_d'],
                                              infer_datetime_format = True)

0         May-2019
1         Feb-2020
2         Feb-2019
3         Oct-2019
4         Feb-2020
            ...   
128279    Dec-2018
128280    Feb-2020
128281    Feb-2020
128282    Feb-2020
128283    Feb-2020
Name: last_pymnt_d, Length: 128284, dtype: object


In [21]:
df_LoanStats['last_pymnt_d_month'] = df_LoanStats['last_pymnt_d'].dt.month
df_LoanStats['last_pymnt_d_year'] = df_LoanStats['last_pymnt_d'].dt.year

df_LoanStats_last_pymnt_date = df_LoanStats[['last_pymnt_d_month',
                                         'last_pymnt_d_year']].copy()

print(df_LoanStats_last_pymnt_date)

        last_pymnt_d_month  last_pymnt_d_year
0                      5.0             2019.0
1                      2.0             2020.0
2                      2.0             2019.0
3                     10.0             2019.0
4                      2.0             2020.0
...                    ...                ...
128279                12.0             2018.0
128280                 2.0             2020.0
128281                 2.0             2020.0
128282                 2.0             2020.0
128283                 2.0             2020.0

[128284 rows x 2 columns]


# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01